In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import math

## Data Gen

In [13]:
# g : R^2 -> {0,1}
# Here, each x \in X are \in R^2
def true_g(X):
    return np.where(np.linalg.norm(X, axis=1) < 3, 0, 1)

# Given g, each x \in X are \in R^3
def true_f(g, X):
    g_result = g(X[:,:2])
    return g_result * (1 - X[:,2]) + (1 - g_result) * X[:,2]

In [14]:
# Number of training points will be split_sizes[0] + split_sizes[1]
def get_train_data(split_sizes):
    n0 = split_sizes[0]
    n1 = split_sizes[1]

    X1_01 = np.around(np.random.uniform(0, 5, size=(n0,2)), 6)
    X1_2 = np.zeros((n0, 1))
    X1 = np.hstack((X1_01, X1_2))

    X2_01 = np.around(np.random.uniform(0, 1, size=(n1,2)), 6)
    X2_2 = np.ones((n1, 1))
    X2 = np.hstack((X2_01, X2_2))

    X_train = np.vstack((X1, X2))
    y_train = true_f(true_g, X_train)
    return X_train, y_train

X_train, y_train = get_train_data([100,100])
train_data = []
for i in range(len(X_train)):
    train_data.append([X_train[i], y_train[i]])
    
trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=20)

## Model

In [15]:
class FeedForward(nn.Module):
    def __init__(self, num_hidden_layers, input_size, hidden_layers_sizes, output_size):
        super(FeedForward, self).__init__()

        self.linears = nn.ModuleList([nn.Linear(input_size, hidden_layers_sizes[0])])
        self.linears.extend([nn.Linear(hidden_layers_sizes[i], hidden_layers_sizes[i+1]) \
            for i in range(num_hidden_layers-1)])
        self.linears.append(nn.Linear(hidden_layers_sizes[-1], output_size))
        self.relu = nn.ReLU()

    def forward(self, x):
        for fc in self.linears[:-1]:
            x = self.relu(fc(x))
        return self.linears[-1](x)

In [49]:
model = FeedForward(3, 3, [5,6,7], 1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

num_epochs = 200
for epoch in range(num_epochs):
    correct = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.float()
        labels = labels.float()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.view(outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        output = (outputs > 0.5).float()
        correct += (output == labels).float().sum()

    acc = correct / len(train_data)
    print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.3f}, Accuracy: {acc:.3f}")


Epoch 0/200, Loss: 0.734, Accuracy: 0.130
Epoch 1/200, Loss: 0.722, Accuracy: 0.130
Epoch 2/200, Loss: 0.710, Accuracy: 0.130
Epoch 3/200, Loss: 0.684, Accuracy: 0.130
Epoch 4/200, Loss: 0.669, Accuracy: 0.130
Epoch 5/200, Loss: 0.645, Accuracy: 0.130
Epoch 6/200, Loss: 0.618, Accuracy: 0.130
Epoch 7/200, Loss: 0.608, Accuracy: 0.130
Epoch 8/200, Loss: 0.601, Accuracy: 0.130
Epoch 9/200, Loss: 0.605, Accuracy: 0.130
Epoch 10/200, Loss: 0.637, Accuracy: 0.130
Epoch 11/200, Loss: 0.542, Accuracy: 0.155
Epoch 12/200, Loss: 0.528, Accuracy: 0.270
Epoch 13/200, Loss: 0.514, Accuracy: 0.630
Epoch 14/200, Loss: 0.472, Accuracy: 0.860
Epoch 15/200, Loss: 0.519, Accuracy: 0.870
Epoch 16/200, Loss: 0.574, Accuracy: 0.870
Epoch 17/200, Loss: 0.503, Accuracy: 0.870
Epoch 18/200, Loss: 0.567, Accuracy: 0.870
Epoch 19/200, Loss: 0.474, Accuracy: 0.870
Epoch 20/200, Loss: 0.388, Accuracy: 0.870
Epoch 21/200, Loss: 0.562, Accuracy: 0.870
Epoch 22/200, Loss: 0.415, Accuracy: 0.870
Epoch 23/200, Loss: 0

Epoch 193/200, Loss: 0.191, Accuracy: 0.870
Epoch 194/200, Loss: 0.284, Accuracy: 0.870
Epoch 195/200, Loss: 0.213, Accuracy: 0.870
Epoch 196/200, Loss: 0.373, Accuracy: 0.870
Epoch 197/200, Loss: 0.470, Accuracy: 0.870
Epoch 198/200, Loss: 0.206, Accuracy: 0.870
Epoch 199/200, Loss: 0.199, Accuracy: 0.870
